# Detekcija Alzheimerove bolesti na MRI slikama mozga

In [98]:
import os
from glob import glob
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
import sklearn.metrics as metrics
import numpy as np

## Učitavanje podataka

In [80]:
labels = ['Mild_Demented', 'Moderate_Demented', 'Non_Demented', 'Very_Mild_Demented']
images = []
Y = []

for label in labels:
    folderPath = os.path.join('Data', 'Dataset', label)
    for im in tqdm(os.listdir(folderPath)):
        images.append(cv2.imread(folderPath + "\\" + im))
        Y.append(label)

100%|██████████| 2240/2240 [00:13<00:00, 161.36it/s]


## Modeli za detekciju Alzheimerove bolesti

In [82]:
no_alzheimer = []
alzheimer = []

for image, y in zip(images, Y):
    if y == 'Non_Demented':
        no_alzheimer.append(image)
    else:
        alzheimer.append(image)

detection_X = no_alzheimer + alzheimer
detection_Y = [0] * len(no_alzheimer) + [1] * len(alzheimer)

Podjela podataka na skup za treniranje i testiranje

In [85]:
X_train, X_test, Y_train, Y_test = train_test_split(detection_X, detection_Y, train_size = 0.75, stratify = Y, shuffle = True, random_state=357)

### 1. SVM

In [90]:
svm_model = SVC(kernel='linear')
X_train = np.array(X_train).reshape(len(X_train), -1)
svm_model.fit(X_train, Y_train)

Y_predicted_1 = svm_model.predict(X_train)

In [91]:
print('Accuracy : ', metrics.accuracy_score(Y_train, Y_predicted_1))
print('Precision : ', metrics.precision_score(Y_train, Y_predicted_1))
print('Recall : ', metrics.recall_score(Y_train, Y_predicted_1))
print('f1 : ', metrics.f1_score(Y_train, Y_predicted_1)) 
print('Confusion matrix :', metrics.confusion_matrix(Y_train, Y_predicted_1))

Accuracy :  1.0
Precision :  1.0
Recall :  1.0
f1 :  1.0
Confusion matrix : [[2410    0]
 [   0 2390]]


In [96]:
X_test = np.array(X_test).reshape(len(X_test), -1)
Y_predicted_2 = svm_model.predict(X_test)

In [101]:
print(metrics.classification_report(Y_test, Y_predicted_2))

              precision    recall  f1-score   support

           0       0.98      0.98      0.98       790
           1       0.98      0.98      0.98       810

    accuracy                           0.98      1600
   macro avg       0.98      0.98      0.98      1600
weighted avg       0.98      0.98      0.98      1600



### 2. 